## Imports

In [1]:
import torch
print(f'PyTorch CUDA is available? {torch.cuda.is_available()}')
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PyTorch CUDA is available? True


## Reproducibility

In [2]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

## Devices available

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

Device: cuda


## Access Dataloader

In [4]:
from pathlib import Path
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset
from typing import List, Tuple

In [5]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

In [6]:
from torchvision import transforms as T, datasets

In [7]:
## Rimettere a 260x260

tsfms_std = T.Compose([
    T.Resize(size=(224, 224)),
    T.ToTensor(),
    # T.Lambda(lambda x: x.flatten()),
])

tsfms_increasing = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomResizedCrop(size=(150, 150), scale=(0.7, 0.9)),
    T.ToTensor(),
    T.Resize(size=(224, 224)),
    # T.Lambda(lambda x: x.flatten()),
])

train_dset = GroceryStoreDataset(
    split="train",
    transform=tsfms_std,
)
increased_train_dst = GroceryStoreDataset(
    split="train",
    transform=tsfms_increasing,
)
val_dset = GroceryStoreDataset(
    split="val",
    transform=tsfms_std,
)
test_dset = GroceryStoreDataset(
    split="test",
    transform=tsfms_std,
)
n_classes = 43
input_dim = len(train_dset[0][0])

increased_dataset = torch.utils.data.ConcatDataset([increased_train_dst,train_dset])
print(len(increased_dataset))

5280


In [8]:
from torch.utils.data import DataLoader

In [9]:
print(len(train_dset))
print(len(val_dset))

2640
296


In [10]:
batch_size = 64

train_dl = DataLoader(
    train_dset,
    batch_size=batch_size,
    shuffle=True
)
val_dl = DataLoader(
    val_dset,
    batch_size=batch_size
)
test_dl = DataLoader(
    test_dset,
    batch_size=batch_size
)

## ResNet Model

In [11]:
from torchvision.models import resnet18, ResNet18_Weights

In [68]:
class Resnet18(torch.nn.Module):
    def __init__(self, n_classes, weights):
        super().__init__()
        self.base_model = resnet18(weights=weights)
        self.base_model.fc = torch.nn.Linear(self.base_model.fc.in_features, n_classes)
    
    def set_requires_grad(self, layer, train):
        for p in layer.parameters():
            p.requires_grad = train
    
    def forward(self, x):
        x = self.base_model(x)
        return x


In [78]:
resnet_model = Resnet18(
    n_classes=n_classes,
    weights=ResNet18_Weights.IMAGENET1K_V1
)

In [79]:
pushed_net = resnet_model.to(device)

In [80]:
## Freeze base model layer (not fc)
for layer in [pushed_net.base_model.conv1, pushed_net.base_model.bn1, pushed_net.base_model.layer1, pushed_net.base_model.layer2, pushed_net.base_model.layer3, pushed_net.base_model.layer4]:
    pushed_net.set_requires_grad(layer, train=False)

### Compile network

In [81]:
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import OneCycleLR

In [82]:
initial_lr = 1e-3
optimizer = AdamW(pushed_net.parameters(), lr=initial_lr, weight_decay=1e-4)
num_epochs = 30
num_steps = num_epochs * len(train_dl)
lr_scheduler = OneCycleLR(optimizer, initial_lr, total_steps=num_steps)

## Train network

In [64]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [83]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, scheduler, val_dl=None, verbose=False):
    best_val_acc = 0
    best_params = []
    best_epoch = -1

    for e in tqdm(range(epochs)):
    # for e in range(epochs):
        model.train()
        # Train
        train_loss = 0
        train_samples = 0
        train_acc = 0
        # running_loss = 0
        # correct = 0
        # total = 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)

            opt.zero_grad()  # clear

            scores = model(imgs)
            loss = F.cross_entropy(scores, labels)

            loss.backward()  # fill
            opt.step()       # Weight Optimizer step
            scheduler.step() # Learning Rate Scheduler step


            # running_loss += loss.item()
            # _, predicted = torch.max(scores.data, 1)
            # total += labels.size(0)
            # correct += (predicted == labels).sum().item()

            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            
        # train_loss = running_loss / len(train_dl)
        # train_acc = 100 * correct / total
        train_acc /= train_samples
        train_loss /= train_samples

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0
            # correct = 0
            # total = 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]

                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()

                    # val_scores = model(imgs)
                    # _, predicted = torch.max(val_scores.data, 1)
                    # total += labels.size(0)
                    # correct += (predicted == labels).sum().item()

                val_acc /= val_samples
                val_loss /= val_samples
                # val_acc = 100 * correct / total

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model2.pth")
                best_epoch = e

        # if verbose and e % 5 == 0:
        if verbose:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))
            # print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss - valid acc {val_acc:.3f}"))

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return best_val_acc, best_params, best_epoch

In [85]:
best_val_acc, best_params, best_epoch = train_loop(
    pushed_net,
    train_dl,
    num_epochs,
    optimizer,
    lr_scheduler,
    val_dl,
    verbose=True
)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0: train loss 3.995 - train acc 0.013 - valid loss 3.847 - valid acc 0.034
Epoch 1: train loss 3.645 - train acc 0.034 - valid loss 3.529 - valid acc 0.047
Epoch 2: train loss 3.286 - train acc 0.075 - valid loss 3.296 - valid acc 0.078
Epoch 3: train loss 2.949 - train acc 0.197 - valid loss 2.976 - valid acc 0.196
Epoch 4: train loss 2.546 - train acc 0.356 - valid loss 2.636 - valid acc 0.314
Epoch 5: train loss 2.120 - train acc 0.495 - valid loss 2.304 - valid acc 0.405
Epoch 6: train loss 1.714 - train acc 0.627 - valid loss 2.036 - valid acc 0.476
Epoch 7: train loss 1.389 - train acc 0.718 - valid loss 1.803 - valid acc 0.500
Epoch 8: train loss 1.151 - train acc 0.776 - valid loss 1.609 - valid acc 0.588
Epoch 9: train loss 0.961 - train acc 0.840 - valid loss 1.496 - valid acc 0.608
Epoch 10: train loss 0.820 - train acc 0.875 - valid loss 1.382 - valid acc 0.618
Epoch 11: train loss 0.715 - train acc 0.902 - valid loss 1.331 - valid acc 0.652
Epoch 12: train loss 0.640

In [84]:
torch.cuda.empty_cache()